# Nome : Alberto Rodrigues Ferreia
# Matrícula : 391278

# Implementação sem a biblioteca Scikit-Learn

In [1]:
import pandas as pd
import numpy as np

# Importando dados já pré-processados pelo R, sem nenhum valor faltante.

In [2]:
dados=pd.read_csv("C://Users//Alberto//Desktop//Trabalhos//DadosDeCarrosRedesNeurais.csv")

In [3]:
dados.head()

,make,fuel-type,aspiration,body-style,engine-location,wheel-base,length,width,height,curb-weight,engine-type,engine-size,fuel-system,bore,stroke,compression-ratio,peak-rpm,highway-mpg,price
0,alfa-romero,gas,std,convertible,front,88.6,168.8,64.1,48.8,2548,dohc or dohcv or l or ohc,130,mfi or mpfi or spdi or spfi,3.47,2.68,9.0,5000.0,27,13495
1,alfa-romero,gas,std,convertible,front,88.6,168.8,64.1,48.8,2548,dohc or dohcv or l or ohc,130,mfi or mpfi or spdi or spfi,3.47,2.68,9.0,5000.0,27,16500
2,alfa-romero,gas,std,hatchback,front,94.5,171.2,65.5,52.4,2823,ohcf or ohcv or rotor,152,mfi or mpfi or spdi or spfi,2.68,3.47,9.0,5000.0,26,16500
3,audi,gas,std,sedan,front,99.8,176.6,66.2,54.3,2337,dohc or dohcv or l or ohc,109,mfi or mpfi or spdi or spfi,3.19,3.40,10.0,5500.0,30,13950
4,audi,gas,std,sedan,front,99.4,176.6,66.4,54.3,2824,dohc or dohcv or l or ohc,136,mfi or mpfi or spdi or spfi,3.19,3.40,8.0,5500.0,22,17450


# Função para as variáveis explicativas que são categóricas com o objetivo de usá-las como variáveis dummy com o método de Regressão Casela de Referência.

In [4]:
#Essa função irá fazer parte da matriz X para cada variável categórica, ela irá percorrer todas as observações dessa variável
# e dizer 1 se ela for de um tipo específico e 0 se não for, e faz por coluna.
def CaselaDeReferencia(x):
    #Criando matriz para cada variável categórica com o número de linhas igual ao número de observações e o número de colunas
    #igual ao número de categorias únicas - 1.
    matriz=np.empty(shape=(len(x),len(np.unique(x))-1),dtype=int)
    #Andando pelas indices das colunas.
    for i in range(matriz.shape[1]):
        #Andando pelo indice das linhas (observações).
        for j in range(len(x)):
            #Se o valor for
            if x[j]==np.unique(x)[1:][i]:
                matriz[j,i]=1
            else:
                matriz[j,i]=0
    return matriz

In [5]:
#Nome das variáveis
dados.columns

Index(['make', 'fuel-type', 'aspiration', 'body-style', 'engine-location',
       'wheel-base', 'length', ' width', 'height', 'curb-weight',
       'engine-type', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'peak-rpm', 'highway-mpg', 'price'],
      dtype='object')

# Irei fazer pelo método dos mínimos quadrados ordinários, pois possuem excelentes propriedades sobre os estimadores de Regressão Linear, essa é a função dos estimadores.

In [6]:
#A função recebe dois parâmetros, x e y para estimar os parâmetros.
def Beta(x,y):
    return(np.linalg.inv((x.T).dot(x)).dot(x.T).dot(y))

In [7]:
#Pegando as variáveis que são numéricas.
DadosNumericos=(dados[['wheel-base', 'length', ' width', 'height','curb-weight','engine-size','bore'
, 'stroke','compression-ratio', 'peak-rpm', 'highway-mpg']].values)

# Ocorreram alguns problemas com as variáveis categóricas e algumas delas serão retiradas e será explicado o motivo.

# A variável make como mostrado abaixo possui várias categórias e algumas dessas categórias existe uma frequência muita baixa dessa categoria e esse é o problema pois na matriz X, utilizando o método de validação cruzada 10-folds pode acontecer de que todas as observações de um certo tipo de categoria esteja toda no conjunto de teste, em algum fold específico, e nos outros 9 folds não tenha nenhuma observação no conjunto de treinamento com aquela tipo e com o método de regressão linear Casela de Referência isso ocassionaria uma coluna de zeros na matrix (X^T*X) e assim não existiria inversa pelo método usual e não teríamos como estimar os parâmetros nesse caso.


In [8]:
np.unique(dados["make"],return_counts=True)

(array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
        'isuzu', 'jaguar', 'mazda', 'mercedes-benz', 'mercury',
        'mitsubishi', 'nissan', 'peugot', 'plymouth', 'porsche', 'renault',
        'saab', 'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object),
 array([ 3,  6,  8,  3,  9, 13,  2,  3, 17,  8,  1, 13, 18, 11,  7,  4,  2,
         6, 12, 32, 12, 11], dtype=int64))

# A variável enfine-location ocorre do mesmo problema tendo o tipo rear com apenas 3 observações e por esse motivo será retirado do modelo.

In [9]:
np.unique(dados["engine-location"],return_counts=True)

(array(['front', 'rear'], dtype=object), array([198,   3], dtype=int64))

In [10]:
#Matriz X e Variável y.
x=(np.column_stack((np.ones(dados.shape[0]),DadosNumericos,
                    CaselaDeReferencia(dados["fuel-type"]),
                    CaselaDeReferencia(dados["aspiration"]),
                    CaselaDeReferencia(dados["body-style"]),
                    CaselaDeReferencia(dados["engine-type"]),
                    CaselaDeReferencia(dados["fuel-system"]))))
y=dados["price"].values

# Estimação dos parâmetros.

In [11]:
while True:
    #Vetor de teste para se ocassionalmente, o que têm poucas chances de ocorrer, pode ser que ainda ainda ocorra o problema
    #de não existir inversa única para a matriz X^T*X com alguma variável categórica, esse vetor será usado para resolver
    #esse problema.
    Teste=[]
    #Vetor de indices dos dados
    a=np.arange(dados.shape[0])
    #Embaralhando o array a aleatoriamente
    np.random.shuffle(a)
    #10-folds da validação cruzada, repartindo em 10 partes iguais tirando a último índice do vetor a que será adicionado 
    #posteriormente.
    Partes=np.hsplit(a[:-1],10)
    s=list(Partes[0])
    #Adicionando o indice da observação que faltava.
    s.append(a[-1])
    Partes[0]=np.array(s)
    #Vetor de medidas para cada fold de treino, será utilizada a medida de erro absoluto médio.
    Metricas=[]
    #Laço para testar em cada um dos dez folds.
    for i in range(len(Partes)):
        #Condição para o problema da variável categórica já explicado anteriormente, se isso acontecer
        #o vetor Teste será acrescentado o conjunto de treino que ocorreu o problema.
        if np.linalg.det((x[np.setdiff1d(a,Partes[i]),:].T).dot(x[np.setdiff1d(a,Partes[i]),:]))==0:
            Teste.append(x[np.setdiff1d(a,Partes[i]),:])
        #Se tudo ocorrer certo ele passará por aqui e estimará os parâmetros.
        else:
            #Beta é a função que criei para estimar os parâmetros, a parte dos parâmetros é o seguinte:
            # o comando np.setdiff1d(a,Partes[i]) significa que vou pegar o conjunto de indices de treinamento, ou seja
            # estou pegando a diferença de conjuntos, a possui todos os indices e Partes[i] possue o conjunto de indices
            #de de treino na i-ésima iteração.
            beta=Beta(x[np.setdiff1d(a,Partes[i]),:],y[np.setdiff1d(a,Partes[i])])
            #Os y's estimados são calculados abaixo.
            ValoresPreditos=x[Partes[i],:].dot(beta)
            #A cada iteração eu ponho a métrica erro absoluto médio na minha lista, para que no final eu tire uma média
            #de todas as métricas de todos os folds.
            Metricas.append(np.mean(np.abs(y[Partes[i]]-ValoresPreditos)))
    #Aqui é consertado o problema de não inversibilidade da matriz X^T*X, pois se haver, haverá uma nova repartição dos folds,
    # e é exatamente por esse motivo que soluciona o problema, novas repartições em cada fold para que não haja um tipo
    #de uma variável categórica que estejam todas as observações no conjunto de teste.
    if len(Teste)==0:
        break
    #Aqui se o problema ocorrer ele fará todo o processo de repartições novamente, até que todo dê certo.
    else:
        continue

In [12]:
#Métrica em cada um dos dez folds.
Metricas

[3077.0515306898305,
 1874.4309234957614,
 1810.2197956694934,
 1682.1860061419866,
 2079.5916220183162,
 2377.709955110954,
 2373.2277754019788,
 3551.648427427967,
 2696.4618709871634,
 2542.3474740373995]

In [13]:
#Avaliação do modelo sem usar a biblioteca Scikit-Learn.
np.mean(Metricas)

2406.4875380980857

# Implementação com a biblioteca Scikit-Learn

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [15]:
while True:
    #Validação cruzada com 10 folds, e o parâmetro shuffle sorteia os indices aleatoriamente para cada fold.
    k=KFold(10,shuffle=True)
    #Vetor de teste para se ocassionalmente, o que têm poucas chances de ocorrer, pode ser que ainda ainda ocorra o problema
    #de não existir inversa única para a matriz X^T*X com alguma variável categórica, esse vetor será usado para resolver
    #esse problema.
    Teste=[]
    #Vetor de metricas para cada fold de teste.
    Metricas2=[]
    for IndiceTreino,IndiceTeste in k.split(x):
        #Classe para o modelo de Regressão Linear.
        modelo=LinearRegression()
        #Condição para o problema da variável categórica já explicado anteriormente, se isso acontecer
        #o vetor Teste será acrescentado o conjunto de treino que ocorreu o problema.
        if np.linalg.det((x[IndiceTreino,:].T).dot(x[IndiceTreino,:]))==0:
            Teste.append(x[IndiceTreino,:])
        #Se tudo ocorrer certo ele passará por aqui e estimará os parâmetros.
        else:
            #Fazendo a repartição do conjunto de treino em nove folds e o conjunto de teste para um fold.
            print("Treino:", IndiceTreino, "Teste:", IndiceTeste,"\n")
            xtreino, xteste= x[IndiceTreino,:], x[IndiceTeste,:]
            ytreino, yteste= y[IndiceTreino], y[IndiceTeste]
            #Treinando o modelo.
            modelo.fit(xtreino,ytreino)
            #Pondo na lista a métrica de um conjunto de teste específico do fold.
            Metricas2.append(np.mean(np.abs(yteste-modelo.predict(xteste))))
    #Aqui é consertado o problema de não inversibilidade da matriz X^T*X, pois se haver, haverá uma nova repartição dos folds,
    # e é exatamente por esse motivo que pode solucionar o problema, novas repartições em cada fold para que não haja um tipo
    #de uma variável categórica que estejam todas as observações no conjunto de teste.
    if len(Teste)==0:
        break
    #Aqui se o problema ocorrer ele fará todo o processo de repartições novamente, até que todo dê certo.
    else:
        continue

Treino: [  0   1   2   3   4   6   7   8   9  10  11  12  13  14  15  16  17  18
  20  21  22  23  24  25  26  27  28  29  30  32  33  34  36  37  38  40
  41  42  44  46  47  48  49  51  52  54  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  72  73  74  75  76  77  78  79  80  81  82
  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 124 125 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 143 144 145 146 147 148 149 150 152 153 154 155 156 158 159
 160 161 162 163 165 166 169 171 172 173 174 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200] Teste: [  5  19  31  35  39  43  45  50  53  55  71 123 126 142 151 157 164 167
 168 170 175] 

Treino: [  0   1   2   3   4   5   7   8  10  11  12  13  14  15  16  19  20  21
  22  23  24  26  27  28  29  31  33  34  35  36  37  38  39  40  41  42
  43  44  4

In [16]:
Metricas2

[2366.130211833871,
 2584.1573787836746,
 3388.973958519211,
 2364.8814069201117,
 2207.809274835298,
 2834.8344872300167,
 2418.201377418179,
 1217.8698326509486,
 2386.4588283592225,
 2432.995100098678]

In [17]:
np.mean(Metricas2)

2420.231185664921